In [135]:
import pandas as pd
key = "2022"
path = f"./asset/副本{key}.xlsx"
outpath = f"./output/{key}.xlsx"
# 读取Excel文件
df_pipei = pd.read_excel(path,sheet_name="Sheet1")
df_zcpm = pd.read_excel(path,sheet_name="Sheet2")

# def findCloest(bankName,df,blackBankNameSet):
#     """
#     找到离 bankName 资产水平最近的银行
#     """
#     # print(f"closet bank to {bankName}")
#     target_row = df.loc[df["id"]==bankName, "资产"].values;
#     # 如果target_row为空, 那就返回空 series
#     if len(target_row)==0:
#         print(f"没有找到银行 {bankName} 的资产")
#         return pd.Series()
#     data_zs= target_row[0]

#     # df=df[df["银行"]!=bankName]
#     df=df[~df['id'].isin(blackBankNameSet)]
#     # # 创建一个 DataFrame
#     # # 计算招商银行资产与其他银行资产的差
#     diff = df["资产"] - data_zs

#     # # 找到与招商银行资产最相近的银行
#     closest_bank = df.loc[diff.abs().idxmin()]
#     return closest_bank


# 在 df_pipei 里找到和 bankCode相同的 bankName
def findBlankWithSameCode(bankCode):
    """
    在 df_pipei 里找到和 bankCode相同的 bankName
    """
    df_pipei_copy =df_pipei.copy();
    df_pipei_copy = df_pipei_copy[df_pipei_copy["证券代码"]==bankCode]
    # print(f"和{bankCode}相同的银行有：\n{df_pipei_copy}")
    return df_pipei_copy["贷款银行"]

# findBlankWithSameCode(2030)

# 获取对应的银行的资产
def getBankInfo(bankName,df):
    """
    获取银行资产
    """
    target_row = df.loc[df["id"]==bankName].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        return pd.Series()
    return target_row[0]
def exportExcel(df):
    """
    导出数据到Excel
    """
    df.to_excel(outpath, sheet_name=key)


# 从 df_pipei 找到 bankName 匹配的行
def findByBlankNameInPipei(bankName):
    target_row = df_pipei.loc[df_pipei["贷款银行"]==bankName].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        return pd.Series()
    return target_row[0]


df_pipei_copy =df_pipei.copy();

In [136]:
def findCloestByZC(bankName,blackBankNameSet):
    df = df_zcpm[20:]
    target_row = df.loc[df["id"]==bankName, "资产"].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        print(f"没有找到银行 {bankName} 的资产")
        return pd.Series()
    data_zs= target_row[0]

    # df=df[df["银行"]!=bankName]
    df=df[~df['id'].isin(blackBankNameSet)]

    # # 计算目标银行资产与其他银行资产的差
    diff = df["资产"] - data_zs
        # # 找到与招商银行资产最相近的银行
    closest_bank = df.loc[diff.abs().idxmin()]

    # df_insameType = df[(df["类型"]==bankType) & (df["资产"]>=data_zs-1000) & (df["资产"]<=data_zs+1000)]
    # # df_insameType = df[(df["类型"]==bankType)]
    # if( len(df_insameType)>=1):
    #     # print(f"df_insameType:\n{df_insameType}")
    #     diff_insampType = df_insameType["资产"] - data_zs
    #     closest_bank=df_insameType.loc[diff_insampType.abs().idxmin()]
    # else:
    #     # print(f"同类型({bankType})中找不到匹配的银行")    
    #     pass

    # print(f"closest_bank:\n {closest_bank}")
    return closest_bank

# 基于指数查询
def  findClosestByZS_20(bankName,blackBankNameSet):
    df = df_zcpm[:20]
    target_row = df.loc[df["id"]==bankName, "指数"].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        print(f"没有找到银行 {bankName} 的资产")
        return pd.Series()
    # print(f"target_row: {target_row}")
    data_zs= target_row[0]

    # df=df[df["银行"]!=bankName]
    zc=df.loc[df["id"]==bankName, "资产"].values[0]
    # print(f"zc: {zc}")
    df=df[(~df['id'].isin(blackBankNameSet)) & (abs(df["资产"]-zc)>=0)]
    # # 计算目标银行资产与其他银行资产的差
    diff = df["指数"] - data_zs
        # # 找到与招商银行资产最相近的银行
    closest_bank = df.loc[diff.abs().idxmin()]
    return closest_bank

def  findClosestByZS(bankName,blackBankNameSet):
    df = df_zcpm
    target_row = df.loc[df["id"]==bankName, "指数"].values;
    # 如果target_row为空, 那就返回空 series
    if len(target_row)==0:
        print(f"没有找到银行 {bankName} 的资产")
        return pd.Series()
    # print(f"target_row: {target_row}")
    data_zs= target_row[0]

    df=df[~df['id'].isin(blackBankNameSet)]
    # # 计算目标银行资产与其他银行资产的差
    diff = df["指数"] - data_zs
        # # 找到与招商银行资产最相近的银行
    closest_bank = df.loc[diff.abs().idxmin()]
    return closest_bank

findClosestByZS("农业银行",{"农业银行"})


id          长沙银行
资产    9047.33493
指数     84.903792
Name: 33, dtype: object

In [139]:
# df_zcpm.iloc[19]["Name"]
data= df_zcpm[df_zcpm["id"]=="农业银行"]
data
# data[0]
# data["Name"]
# findClosestByZS("南京银行",{"南京银行"})
# findCloestByZC("南京银行",{"南京银行"})
# df_zcpm[:1]

,id,资产,指数
2,农业银行,339275.33,84.303976


In [141]:
def run():
    df_pipei_copy = []
    df_pipei_copy =df_pipei.copy();
    bankName_top = set(df_zcpm.iloc[:20,0])
    print(f"top bank name: {bankName_top}")
    # iters= df_pipei.iloc[:10].
    # iterrows()
    iters= df_pipei.iterrows()
    for index,row in iters:
        bankName = row["贷款银行"]
        bankCode = row["证券代码"]
        # 如果bankName是空串或者bankCod 是空串 那就 continue
        if bankName=="" or bankCode=="":
            continue
        # print(bankName:bankName)
        row_bankinfo = getBankInfo(bankName,df_zcpm)
        # print(f"row_bankType:{row_bankinfo}")
        if len(row_bankinfo)<1:
            print(f"没有在资产配置表里找到{bankName}对应的银行")
            continue
        row_bankType=row_bankinfo[2]
        res= findClosestByZS(bankName,findBlankWithSameCode(bankCode))
        # if bankName in bankName_top:
        #     # print(f"in top bank name: {bankName}")
        #     res= findClosestByZS(bankName,findBlankWithSameCode(bankCode))
        #     if(len(res)==0):    
        #         res= findCloestByZC(bankName,findBlankWithSameCode(bankCode))
        # else:
        #     res= findCloestByZC(bankName,findBlankWithSameCode(bankCode))
        if(len(res)==0):
            print(f"找不到{bankName}最近的元素")
            continue
        _name=res["id"]
        # print(f"{bankName}对应的银行:{_name}")
        df_pipei_copy.loc[index,"当前银行资产"]=row_bankinfo[1]
        df_pipei_copy.loc[index,"当前银行指数"]=row_bankinfo[2]
        #修改 df_pipei_copy 的 index 行为 [...rowres[0]res[1]]
        df_pipei_copy.loc[index,"最近银行"]=_name
        df_pipei_copy.loc[index,"最近银行资产"]=res["资产"]
        df_pipei_copy.loc[index,"最近银行指数"]=res["指数"]
        info = findByBlankNameInPipei(_name)
        if len(info)==0:
            # print(f"在配置表里 没有找到银行 [{_name}] 的信息")
            continue
        # print(info)
        df_pipei_copy.loc["index最近银行代码"]= info[2]
    print(f"前十名 {df_pipei_copy[:10]}")
    exportExcel(df_pipei_copy)


for key in ["2014","2015","2016","2017","2018","2019","2020","2021","2022"]:
    path = f"./asset/副本{key}.xlsx"
    outpath = f"./output/{key}.xlsx"
    # 读取Excel文件
    df_pipei = pd.read_excel(path,sheet_name="Sheet1")
    df_zcpm = pd.read_excel(path,sheet_name="Sheet2")
    run()

top bank name: {'中信银行', '上海银行', '广发银行', '北京银行', '华夏银行', '招商银行', '江苏银行', '工商银行', '交通银行', '兴业银行', '建设银行', '中国银行', '恒丰银行', '浙商银行', '平安银行', '光大银行', '民生银行', '农业银行', '浦发银行', '邮储银行'}
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到银行 对应的银行
没有在资产配置表里找到贵阳银行对应的银行
没有在资产配置表里找到九江银行对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到九江银行对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到九江银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到银行对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配置表里找到 对应的银行
没有在资产配